# 2.0 Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = '../backend/data/cleaned_dataset.csv'
df = pd.read_csv(file_path)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28352 entries, 0 to 28351
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   track_id                   28352 non-null  object 
 1   track_name                 28352 non-null  object 
 2   track_artist               28352 non-null  object 
 3   track_popularity           28352 non-null  int64  
 4   track_album_id             28352 non-null  object 
 5   track_album_name           28352 non-null  object 
 6   track_album_release_date   28352 non-null  object 
 7   playlist_name              28352 non-null  object 
 8   playlist_id                28352 non-null  object 
 9   playlist_genre             28352 non-null  object 
 10  playlist_subgenre          28352 non-null  object 
 11  danceability               28352 non-null  float64
 12  energy                     28352 non-null  float64
 13  key                        28352 non-null  int

In [3]:
df_eda = df.set_index('track_id')
df_numeric = df_eda.select_dtypes(include=['int64', 'float64'])